# Project Pandas: índice de acciones realizadas:

1. Importamos el archivo y hacemos una primera inspección del dataframe.

2. Inspeccionamos los valores nulos de cada columna.

3. Definimos drop_cols como las columnas con más de 3.000 registros nulos y eliminamos las columnas en drop_cols.

4. Eliminamos columnas con información poco relevante o duplicada:
    
            Date -> duplicada 
            Name -> poco relevante
            pdf -> duplicada
            href -> duplicada
            Case Number.1 -> duplicada
            Case Number.2 -> duplicada
            original order -> poco relevante
            Species -> baja calidad
    
    
5. Limpiamos la columna Case Number.

6. Visualizamos la distribución de los incidentes por años y eliminamos los registros con valores inferiores a 1890 para 'Year'.

7. Observamos los valores de la columna 'Type' y unificamos las categorías 'Boat' y 'Boating'.

8. Aplicamos mayúsculas a los valores del atributo 'Country' y eliminamos los espacios a principio y final del string.

9. Eliminamos los espacios a principio y final del string de las columnas 'Area' y 'Location'.

10. En la columna 'Sex', recategorizamos los valores singulares.

11. En la columna 'Age', seleccionamos los dos primeros digítos de cada campo y descartamos el resto (se generan algunos errores como en el caso de registros con múltiples edades o registros con meses en lugar de años). Después, convertimos el atributo a numérico.

12. En la columna 'FATAL (Y/N)', recategorizamos los valores singulares.

13. Generamos la edad media por país y agregamos la columna al dataframe.

14. Renombramos las columnas.

15. Reordenamos las columnas (para juntar 'Age' y 'Avg age').

16. Exportamos el dataset limpio como un archivo csv.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

### 1. Importamos el archivo y hacemos una primera inspección del dataframe:

In [2]:
data = pd.read_csv('GSAF5.csv', engine = 'python')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 24 columns):
Case Number               5992 non-null object
Date                      5992 non-null object
Year                      5992 non-null int64
Type                      5992 non-null object
Country                   5949 non-null object
Area                      5590 non-null object
Location                  5496 non-null object
Activity                  5465 non-null object
Name                      5792 non-null object
Sex                       5425 non-null object
Age                       3311 non-null object
Injury                    5965 non-null object
Fatal (Y/N)               5973 non-null object
Time                      2779 non-null object
Species                   3058 non-null object
Investigator or Source    5977 non-null object
pdf                       5992 non-null object
href formula              5991 non-null object
href                      5989 non-null object
C

In [3]:
data.head(1)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN


### 2. Inspeccionamos los valores nulos de cada columna:

In [4]:
null_cols = data.isnull().sum()
null_cols [null_cols > 0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

### 3. Definimos drop_cols como las columnas con más de 3.000 registros nulos y eliminamos las columnas en drop_cols:

In [5]:
drop_cols = list(null_cols[null_cols > 3000].index)
drop_cols

['Time', 'Unnamed: 22', 'Unnamed: 23']

In [6]:
data1 = data.drop(drop_cols, axis=1)
data1.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Species ',
       'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

### 4. Eliminamos columnas con información poco relevante o duplicada:

In [7]:
data1 = data1.drop(['Date','Name', 'Species ', 'pdf', 'href','Case Number.1',
                    'Case Number.2', 'original order'], axis=1)

In [8]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 13 columns):
Case Number               5992 non-null object
Year                      5992 non-null int64
Type                      5992 non-null object
Country                   5949 non-null object
Area                      5590 non-null object
Location                  5496 non-null object
Activity                  5465 non-null object
Sex                       5425 non-null object
Age                       3311 non-null object
Injury                    5965 non-null object
Fatal (Y/N)               5973 non-null object
Investigator or Source    5977 non-null object
href formula              5991 non-null object
dtypes: int64(1), object(12)
memory usage: 608.7+ KB


### 5. Limpiamos la columna Case Number:

In [9]:
data1['Case Number'].replace(regex = True, inplace = True, 
                             to_replace = r'[A-Za-z]', value ='')

data1['Case Number'] = data1['Case Number'].str.replace(r'[.]$', '')

In [10]:
data1.head(2)

,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Investigator or Source,href formula
0,2016.09.18,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...


### 6. Visualizamos la distribución de los incidentes por años y eliminamos los registros con valores inferiores a 1890 para 'Year':

In [11]:
Counter(data1['Year'])

Counter({2016: 103,
         2015: 139,
         2014: 125,
         2013: 122,
         2012: 117,
         2011: 128,
         2010: 101,
         2009: 120,
         2008: 121,
         2007: 112,
         2006: 103,
         2005: 103,
         2004: 92,
         2003: 92,
         2002: 88,
         2001: 92,
         2000: 97,
         1999: 65,
         1998: 65,
         1997: 57,
         1996: 61,
         1995: 76,
         1984: 41,
         1994: 56,
         1993: 56,
         1992: 56,
         1991: 38,
         1990: 38,
         1989: 53,
         1969: 30,
         1988: 55,
         1987: 35,
         1986: 39,
         1985: 37,
         1983: 50,
         1982: 40,
         1981: 49,
         1980: 35,
         1979: 25,
         1978: 25,
         1977: 26,
         1976: 39,
         1975: 49,
         1974: 38,
         1973: 27,
         1972: 35,
         1971: 28,
         1970: 42,
         1968: 46,
         1967: 48,
         1966: 58,
         1965: 51,


In [12]:
data1 = data1[data1.Year >= 1890]

In [13]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5467 entries, 0 to 5467
Data columns (total 13 columns):
Case Number               5467 non-null object
Year                      5467 non-null int64
Type                      5467 non-null object
Country                   5441 non-null object
Area                      5152 non-null object
Location                  5078 non-null object
Activity                  5033 non-null object
Sex                       4948 non-null object
Age                       3243 non-null object
Injury                    5448 non-null object
Fatal (Y/N)               5457 non-null object
Investigator or Source    5454 non-null object
href formula              5466 non-null object
dtypes: int64(1), object(12)
memory usage: 598.0+ KB


### 7. Observamos los valores de la columna 'Type' y unificamos las categorías Boat y Boating:

In [14]:
data1['Type'].value_counts()

Unprovoked      3993
Provoked         536
Invalid          464
Boat             194
Sea Disaster     187
Boating           93
Name: Type, dtype: int64

In [15]:
data1['Type'] = data['Type'].str.replace('Boating', 'Boat')

In [16]:
data1['Type'].value_counts()

Unprovoked      3993
Provoked         536
Invalid          464
Boat             287
Sea Disaster     187
Name: Type, dtype: int64

### 8. Aplicamos mayúsculas a los valores del atributo 'Country' y eliminamos los espacios a principio y final del string:

In [17]:
data1['Country'] = data1['Country'].str.upper()

In [18]:
data1['Country'] = data1['Country'].str.lstrip()
data1['Country'] = data1['Country'].str.rstrip()

### 9. Eliminamos los espacios a principio y final del string de las columnas 'Area' y 'Location':

In [19]:
data1['Area'] = data1['Area'].str.lstrip()
data1['Area'] = data1['Area'].str.rstrip()

In [20]:
data1['Location'] = data1['Location'].str.lstrip()
data1['Location'] = data1['Location'].str.rstrip()

### 10. En la columna 'Sex', recategorizamos los valores singulares:

In [21]:
Counter(data1['Sex '])

Counter({'M': 4387, nan: 519, 'F': 556, 'M ': 2, 'lli': 1, 'N': 1, '.': 1})

In [22]:
data1['Sex '] = data1['Sex '].str.rstrip()
data1['Sex '] = data1['Sex '].replace('lli', None)
data1['Sex '] = data1['Sex '].replace('N', None)
data1['Sex '] = data1['Sex '].replace('.', None)

In [23]:
Counter(data1['Sex '])

Counter({'M': 4391, nan: 519, 'F': 557})

### 11. En la columna 'Age', seleccionamos los dos primeros digítos de cada campo y descartamos el resto (se generan algunos errores como en el caso de registros con múltiples edades o registros con meses en lugar de años). Después, convertimos el atributo a numérico.

In [24]:
Counter(data1['Age'])

Counter({'16': 132,
         '36': 40,
         '43': 43,
         nan: 2224,
         '60s': 1,
         '51': 19,
         '50': 31,
         '12': 68,
         '9': 35,
         '22': 110,
         '25': 100,
         '37': 36,
         '20': 133,
         '49': 27,
         '15': 131,
         '21': 112,
         '40': 47,
         '72': 1,
         '18': 140,
         '29': 74,
         '31': 50,
         '11': 31,
         '10': 51,
         '59': 11,
         '42': 36,
         '34': 45,
         '35': 63,
         '19': 134,
         '6': 13,
         '27': 76,
         '64': 4,
         '60': 13,
         '23': 78,
         '52': 33,
         '13': 87,
         '57': 14,
         '48': 25,
         '39': 36,
         '24': 103,
         '26': 79,
         '69': 8,
         '46': 25,
         'Teen': 8,
         '41': 34,
         '45': 32,
         '65': 4,
         '38': 47,
         '71': 3,
         '32': 64,
         '58': 12,
         '28': 76,
         '54': 14,
        

In [25]:
data1['Age'] = data1['Age'].str.extract(r'^(\d{2})', expand = False)

In [26]:
data1['Age'].value_counts()

17    145
18    143
20    140
19    134
16    133
     ... 
81      1
86      1
73      1
72      1
67      1
Name: Age, Length: 72, dtype: int64

In [27]:
data1["Age"] = pd.to_numeric(data1["Age"])

In [28]:
data1["Age"].mean()

27.941747572815533

### 12. En la columna 'FATAL (Y/N)', recategorizamos los valores singulares:

In [29]:
Counter(data1['Fatal (Y/N)'])

Counter({'N': 4085,
         'Y': 1278,
         nan: 10,
         'UNKNOWN': 83,
         ' N': 8,
         'F': 1,
         'N ': 1,
         '#VALUE!': 1})

In [30]:
data1['Fatal (Y/N)'] = data1['Fatal (Y/N)'].replace('F', 'UNKNOWN')
data1['Fatal (Y/N)'] = data1['Fatal (Y/N)'].replace('#VALUE!', 'UNKNOWN')
data1['Fatal (Y/N)'] = data1['Fatal (Y/N)'].str.rstrip()
data1['Fatal (Y/N)'] = data1['Fatal (Y/N)'].str.lstrip()

In [31]:
Counter(data1['Fatal (Y/N)'])

Counter({'N': 4094, 'Y': 1278, nan: 10, 'UNKNOWN': 85})

In [32]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5467 entries, 0 to 5467
Data columns (total 13 columns):
Case Number               5467 non-null object
Year                      5467 non-null int64
Type                      5467 non-null object
Country                   5441 non-null object
Area                      5152 non-null object
Location                  5078 non-null object
Activity                  5033 non-null object
Sex                       4948 non-null object
Age                       3090 non-null float64
Injury                    5448 non-null object
Fatal (Y/N)               5457 non-null object
Investigator or Source    5454 non-null object
href formula              5466 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 598.0+ KB


### 13. Generamos la edad media por país y agregamos la columna al dataframe:

In [33]:
avg_age = data1.groupby('Country', as_index = False)['Age'].mean()

In [34]:
data1.pivot_table(index='Country', values = 'Age', aggfunc = np.mean)

,Age
Country,
ADMIRALTY ISLANDS,26.000000
AMERICAN SAMOA,25.666667
ANDAMAN / NICOBAR ISLANDAS,32.000000
ARGENTINA,18.000000
ARUBA,58.000000
...,...
VANUATU,23.666667
VENEZUELA,37.000000
VIETNAM,42.166667


In [35]:
data1 = pd.merge(data1, avg_age, on='Country')
data1['Age_y'] = data1['Age_y'].round()
data1.head(1)

,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Age_x,Injury,Fatal (Y/N),Investigator or Source,href formula,Age_y
0,2016.09.18,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16.0,Minor injury to thigh,N,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,28.0


### 14. Renombramos las columnas:

In [36]:
data1 = data1.rename(columns={'Case Number':'Date',
                             'Sex ':'Sex',
                             'Age_x':'Age',
                             'Fatal (Y/N)': 'Fatal',
                             'Age_y':'Avg age'})

### 15. Reordenamos las columnas (para juntar 'Age' y 'Avg age'):

In [37]:
column_order = ['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Sex', 'Age', 'Avg age','Injury', 'Fatal', 'Investigator or Source',
       'href formula']

data1 = data1[column_order]
    
data1.head(1)

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Avg age,Injury,Fatal,Investigator or Source,href formula
0,2016.09.18,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16.0,28.0,Minor injury to thigh,N,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...


### 16. Exportamos el dataset limpio como un archivo csv:

In [38]:
data1.to_csv('data1.csv', index = False)

In [39]:
# Comparación datos originales vs datos finales:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 24 columns):
Case Number               5992 non-null object
Date                      5992 non-null object
Year                      5992 non-null int64
Type                      5992 non-null object
Country                   5949 non-null object
Area                      5590 non-null object
Location                  5496 non-null object
Activity                  5465 non-null object
Name                      5792 non-null object
Sex                       5425 non-null object
Age                       3311 non-null object
Injury                    5965 non-null object
Fatal (Y/N)               5973 non-null object
Time                      2779 non-null object
Species                   3058 non-null object
Investigator or Source    5977 non-null object
pdf                       5992 non-null object
href formula              5991 non-null object
href                      5989 non-null object
C

In [40]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5441 entries, 0 to 5440
Data columns (total 14 columns):
Date                      5441 non-null object
Year                      5441 non-null int64
Type                      5441 non-null object
Country                   5441 non-null object
Area                      5142 non-null object
Location                  5071 non-null object
Activity                  5010 non-null object
Sex                       4924 non-null object
Age                       3086 non-null float64
Avg age                   5371 non-null float64
Injury                    5422 non-null object
Fatal                     5432 non-null object
Investigator or Source    5428 non-null object
href formula              5440 non-null object
dtypes: float64(2), int64(1), object(11)
memory usage: 637.6+ KB
